In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import requests
import json
import ast
import pprint
import numpy as np

from datetime import datetime

pd.set_option("display.float_format", lambda x: f"{x :.2f}")
pd.set_option("display.max_columns", None)
from get_dataframes import GetDataframes


from tools import import_config, import_datasets, logging

In [ ]:
import requests
import json
import ast
import pprint


api_key = "fe4a6f12753fa6c12b0fc0253b5e667f"
language = "fr-FR"
imdb_id = 120

params = {
    "api_key": api_key,
    "include_adult": "False",
    "language": language,
    # "append_to_response": "keywords,credits,videos"
}

find = False

# if find:
#     params.update(
#         {"external_source" : "imdb_id"}
#     )
#     find_id = "https://api.themoviedb.org/3/find/"
# else:
base_url = "https://api.themoviedb.org/3/movie/"
person_url = "https://api.themoviedb.org/3/person/"

url = f"{base_url}{imdb_id}"

r = requests.get(url, params=params)
data = r.json()

cc = [
    # ("keywords", "keywords", "name", "keywords"),
    ("genres", "genres", "name"),
    ("spoken_languages", "spoken_languages", "iso_639_1"),
    ("production_companies_name", "production_companies", "name"),
    ("production_countries", "production_countries", "iso_3166_1"),
    (
        "production_companies_country",
        "production_companies",
        "origin_country",
    ),
]

for k, c, v, *sb in cc:
    if sb:
        data[k] = [k[v] for k in data[c][sb[0]]]
    else:
        data[k] = [k[v] for k in data[c]]

# data["keywords"] = [n["name"] for n in data["keywords"]["keywords"][:4]]
# data["actors"] = [n["name"] for n in data["credits"]["cast"] if n["known_for_department"] == "Acting" and n["order"] <= 2]
# data["director"] = [n["name"] for n in data["credits"]["crew"] if n["job"] == "Director"]
# data["url"] = f"https://www.imdb.com/title/{data['imdb_id']}"
# data["image"] = f"https://image.tmdb.org/t/p/w300_and_h450_bestv2{data['poster_path']}"
# data["youtube"] = [f"https://www.youtube.com/watch?v={n['key']}" for n in data["videos"]["results"]]

to_pop = (
    "belongs_to_collection",
    "production_companies",
)
for tp in to_pop:
    data.pop(tp)

data["release_date"] = data["release_date"][:4]
pprint.pprint(data)

In [ ]:
import pandas as pd
import ast

df = pd.read_csv("movies_datasets/tmdb_full.csv", low_memory=False)


def decode_clean(item: str) -> str:
    return (
        item.replace("[", "")
        .replace("]", "")
        .replace("'", "")
        .replace('"', "")
    )


def safe_literal_eval(s):
    try:
        return ast.literal_eval(s) if isinstance(s, str) else s
    except:
        return []


col = [
    "genres",
    "spoken_languages",
    "production_companies_name",
    "production_countries",
    "production_companies_country",
]

for c in col:
    df[c] = df[c].apply(
        lambda x: [
            decode_clean(item)
            for item in safe_literal_eval(x)
            if isinstance(item, str) and item.strip()
        ]
        if isinstance(x, str)
        else []
    )

In [3]:
def fetch_actors_bio(imdb_id):
    api_key = "fe4a6f12753fa6c12b0fc0253b5e667f"
    language = "fr-FR"
    params = {
        "api_key": api_key,
        "include_adult": "False",
        "language": language,
        "append_to_response": "combined_credits",
    }
    base_url = "https://api.themoviedb.org/3/person/"
    url_image = "https://image.tmdb.org/t/p/w300_and_h450_bestv2"
    url_youtube = "https://www.youtube.com/watch?v="
    url = f"{base_url}{imdb_id}"
    r = requests.get(url, params=params)
    data = r.json()

    data["image"] = f"{url_image}{data['profile_path']}"
    top_credits = sorted(
        (
            n for n in data["combined_credits"]["cast"]
            if n["media_type"] == "movie" and n["order"] <= 3
            and all(genre not in n["genre_ids"] for genre in [99, 16, 10402]) # 99: Documentaire, 16: Animation, 10402: Musique
        ),
        key=lambda x: (-x['popularity'], -x['vote_average'], -x["vote_count"])
    )[:8]
    data["top_5"] = [n["title"] for n in top_credits]
    data["top_5_images"] = [f"{url_image}{n['poster_path']}" for n in top_credits]
    data["top_5_TMDb_ids"] = [n['id'] for n in top_credits]

    to_pop = (
        "adult",
        "also_known_as",
        "gender",
        "homepage",
        "profile_path",
        "combined_credits",
        "known_for_department",
    )
    for tp in to_pop:
        data.pop(tp)
    # pprint.pprint(data)
    return data


fetch_actors_bio(2037)


{'biography': 'Cillian Murphy est un acteur irlandais de cinéma et de théâtre. Il est souvent noté par les critiques pour ses performances caméléoniques dans divers rôles et yeux bleus distinctifs.\n\nOriginaire de Cork, Murphy a commencé sa carrière en tant que musicien de rock. Après avoir refusé un contrat d\'enregistrement, il fait ses débuts professionnels dans la pièce Disco Pigs en 1996. Il joue ensuite dans des films et des productions théâtrales irlandaises et britanniques à la fin des années 1990 et au début des années 2000. le héros du film post-apocalyptique 28 Days Later. Les rôles les plus connus de Murphy sont les méchants dans deux superproductions de 2005: l\'épouvantail dans le film de super-héros Batman Begins, et Jackson Rippner dans le thriller Red Eye. Viennent ensuite deux rôles principaux contrastés, largement acclamés: sa performance nommée Golden Globe Award en tant que paria transgenre "Kitten" en 2005 sur Breakfast on Pluto et en tant que révolutionnaire irl

In [ ]:

# if find:
#     params.update(
#         {"external_source" : "imdb_id"}
#     )
#     find_id = "https://api.themoviedb.org/3/find/"
# else:


api_key = "fe4a6f12753fa6c12b0fc0253b5e667f"
language = "fr-FR"
# imdb_id = 120
imdb_id = 2037

params = {
    "api_key": api_key,
    "include_adult": "False",
    "language": language,
    "append_to_response": "combined_credits",
}

find = False

base_url = "https://api.themoviedb.org/3/person/"
url_image = "https://image.tmdb.org/t/p/w300_and_h450_bestv2"
url_youtube = "https://www.youtube.com/watch?v="
# url = f"{base_url}{imdb_id}/movie_credits"
url = f"{base_url}{imdb_id}"

r = requests.get(url, params=params)
data = r.json()

data["image"] = f"{url_image}{data['profile_path']}"
top_credits = sorted(
    (
        n for n in data["combined_credits"]["cast"]
        if n["media_type"] == "movie" and n["order"] <= 3
        and all(genre not in n["genre_ids"] for genre in [99, 16, 10402]) # 99: Documentaire, 16: Animation, 10402: Musique
    ),
    key=lambda x: (-x['popularity'], -x['vote_average'], -x["vote_count"])
)[:8]
data["top_5"] = [n["title"] for n in top_credits]
data["top_5_images"] = [f"{url_image}{n['poster_path']}" for n in top_credits]
data["top_5_TMDb_ids"] = [n['id'] for n in top_credits]

to_pop = (
    "adult",
    "also_known_as",
    "gender",
    "homepage",
    "profile_path",
    "combined_credits",
    "known_for_department",
    # "imdb_id",
    # "popularity",
)
for tp in to_pop:
    data.pop(tp)

# data["release_date"] = data["release_date"][:4]
pprint.pprint(data)
# pprint.pprint(top_credits)
# Supposons que `data` est votre liste de dictionnaires contenant les données des films


In [ ]:
tmd.dropna(subset=["release_date"], inplace=True)
tmd.reset_index(drop="index", inplace=True)

In [ ]:
tmd["release_date"].info()

In [ ]:
tmd["release_date"] = pd.to_datetime(tmd["release_date"])
tmd["release_date"] = tmd["release_date"].apply(lambda x: x.strftime("%Y"))

In [ ]:
config = tmd[""]
tmd[tmd["adult"]]

In [ ]:
import requests
import json
import ast
import pprint
import pandas as pd

api_key = "fe4a6f12753fa6c12b0fc0253b5e667f"
language = "en-US"
imdb_id = 19995

params = {
    "api_key": api_key,
    "include_adult": "False",
    "language": language,
    "append_to_response": "keywords,credits,videos",
}

base_url = "https://api.themoviedb.org/3/movie/"

url = f"{base_url}{imdb_id}"

r = requests.get(url, params=params)
data = r.json()

cleaning = (
    ("keywords", "keywords", "name", "keywords"),
    ("genres", "genres", "name"),
    ("spoken_languages", "spoken_languages", "iso_639_1"),
    ("production_companies_name", "production_companies", "name"),
    ("production_countries", "production_countries", "iso_3166_1"),
)
for key, category, value, *subkey in cleaning:
    if subkey:
        data[key] = [k[value] for k in data[category][subkey[0]]]
    else:
        data[key] = [k[value] for k in data[category]]
data.pop("homepage")
data.pop("belongs_to_collection")
data.pop("production_companies")
pprint.pprint(data)

df = pd.DataFrame([data])
df

In [ ]:
cleaning = (
    ("name", "keywords", "keywords"),
    ("name", "genres"),
    ("iso_639_1", "spoken_languages"),
    ("name", "production_companies"),
    ("iso_3166_1", "production_countries"),
)

for key, category, *subkey in cleaning:
    if subkey:
        data[category] = [item[key] for item in data[category][subkey[0]]]
    else:
        data[category] = [item[key] for item in data[category]]

In [ ]:
def clean_fake_list(s):
    t = []
    for i in ast.literal_eval(s):
        print(i)


df["keywords"] = df["keywords"].apply(clean_fake_list)

In [ ]:
tmd = import_datasets("clean_datasets/tmdb_full.parquet", "parquet")

In [ ]:
tmd

In [ ]:
from datetime import datetime
from dateutil import parser
from time import time


now = datetime.now()


def date_to_ts(d):
    return int(
        parser.parse(d).replace(tzinfo=datetime.timezone.utc).timestamp()
        * 1000
    )


def utc_ms() -> float:
    return datetime.utcnow().timestamp() * 1000


now = str(datetime.now())


def date_to_ts_now():
    return int(datetime.timestamp(datetime.now()))


now

In [ ]:
config = import_config()
datas = GetDataframes(config)
link = "movies_cleaned"
df = datas.get_dataframes(link, True)

In [ ]:
df[df["titre_id"].str.contains("tt5013056")]

In [ ]:
synthese_docstrings = (
    "Compréhension de la Méthode : Comprendre la méthode, ses paramètres, "
    + "processus internes, et sorties. "
    + "Utilisation du Format Numpy : Utiliser le format de docstring Numpy pour "
    + "sa clarté dans la description des paramètres et des valeurs de retour. "
    + "Structure du Docstring : Inclure : "
    + "- Brève Description : Résumer la fonction de la méthode en une ou deux lignes. "
    + "- Parameters : Lister et décrire tous les paramètres. "
    + "- Returns : Expliquer les valeurs retournées par la méthode. "
    + "Rédaction Précise et Limitée : Être concis et précis, respecter la limite "
    + "de 75 caractères (espace compris) par ligne selon PEP8. "
    + "Terminologie Technique Adéquate : Utiliser un langage technique approprié, "
    + "par exemple des termes spécifiques aux DataFrames pour les méthodes les concernant. "
)
print(synthese_docstrings)

In [ ]:
t = "Je suis une IA spécialisée en Python scientifique, avec une expertise dans la rédaction de code et de documentation en français. Mon approche consiste à analyser et comprendre le code en profondeur avant de rédiger des docstrings. Ces dernières sont créées dans le style numpy, visant à être concises, claires, et faciles à comprendre. Je respecte scrupuleusement les règles du PEP8, notamment la limite de 65 caractères par ligne pour garantir une excellente lisibilité. Mon objectif est de fournir des explications détaillées non seulement sur le fonctionnement des fonctions, mais aussi sur leur rôle et leur intégration dans un script, assurant ainsi une compréhension complète pour les utilisateurs."
t = t.split(".")
t
p = (
    "Je suis une IA spécialisée en Python scientifique, avec une expertise dans la rédaction de code et de documentation en français"
    + "Mon approche consiste à analyser et comprendre le code en profondeur avant de rédiger des docstrings"
    + "Ces dernières sont créées dans le style numpy en anglais c'est a dire (Methods, Atributs, Parameters, Returns, Raises, Notes etc...), visant à être concises, claires, et faciles à comprendre"
    + "Je respecte scrupuleusement les règles du PEP8, notamment la limite de 65 caractères par ligne pour garantir une excellente lisibilité"
    + "Mon objectif est de fournir des explications détaillées non seulement sur le fonctionnement des fonctions"
    + "mais aussi sur leur rôle et leur intégration dans un script, assurant ainsi une compréhension complète pour les utilisateurs"
)

In [ ]:
ai_description = (
    "La documentation de la fonction doit débuter par une explication concise et "
    "directe de son but principal, évitant l'usage d'un titre 'description'. Ensuite, "
    "chaque paramètre est décrit individuellement, mentionnant son type, "
    "son rôle et toute contrainte ou valeur par défaut pertinente. Si la fonction "
    "renvoie quelque chose, la nature de cette valeur de retour est également "
    "détaillée, en précisant son type et ce qu'elle représente dans le contexte de "
    "la fonction. Des sections supplémentaires peuvent être ajoutées pour évoquer "
    "des exceptions potentielles levées par la fonction et des notes supplémentaires, "
    "si nécessaire. Cependant, il est important de garder ces sections courtes et "
    "pertinentes. Tout au long de la documentation, le style d'écriture doit être clair, "
    "concis et conforme aux normes PEP8, en veillant à ce que chaque ligne ne dépasse "
    "pas 70 caractères. Cela facilite la lisibilité et la maintenance du code. Le formatage "
    "Black, connu pour son style épuré et sa cohérence, doit être appliqué pour assurer "
    "une présentation uniforme et professionnelle du document."
)

In [ ]:
name = "machine_learning"

logging.info(f"Creating {name} dataframe...")
tmdb_l = "clean_datasets/tmdb_updated.parquet"
actors_l = "clean_datasets/actors_movies.parquet"
directors_l = "clean_datasets/directors_movies.parquet"
movies_l = "clean_datasets/movies_cleaned.parquet"

tmdb = import_datasets(tmdb_l, "parquet")
actors = import_datasets(actors_l, "parquet")
directors = import_datasets(directors_l, "parquet")
movies = import_datasets(movies_l, "parquet")

col_to_keep = [
    "titre_id",
    "titre_str",
    "titre_genres",
    "rating_avg",
    "rating_votes",
]
movies = movies[col_to_keep]

In [ ]:
directors_list_id = directors["titre_id"]
condi = movies["titre_id"].isin(directors_list_id)
condi2 = actors["titre_id"].isin(directors_list_id)
movies = movies[condi]
actors = actors[condi2]

actors_list_id = actors["titre_id"]
condi = movies["titre_id"].isin(actors_list_id)
condi2 = directors["titre_id"].isin(actors_list_id)
movies = movies[condi]
directors = directors[condi2]

In [ ]:
print(len(directors_list_id))
print(len(actors_list_id))
print(len(movies))

In [ ]:
col_to_keep = ["imdb_id", "overview"]
tmdb = tmdb[col_to_keep]

col_to_keep = [
    "titre_id",
    "person_name",
    # "person_index"
]
actors = actors[col_to_keep]

col_to_keep = [
    "titre_id",
    "person_name",
    # "person_index"
]
directors = directors[col_to_keep]

actors.loc[:, "person_name"] = actors["person_name"].str.split(", ")
directors.loc[:, "person_name"] = directors["person_name"].str.split(", ")

person_name = actors.groupby("titre_id")["person_name"].sum().reset_index()
person_list = person_name["person_name"].to_list()

directors_name = (
    directors.groupby("titre_id")["person_name"].sum().reset_index()
)
directors_list = directors_name["person_name"].to_list()

movies["actors"] = person_list
movies["directors"] = directors_list

In [ ]:
print(len(directors_list_id))
print(len(actors_list_id))
print(len(movies))

In [ ]:
logging.info(f"Merging {name} dataframe...")
ml_df = pd.merge(movies, tmdb, left_on="titre_id", right_on="imdb_id")

logging.info(f"Droping NaN {name} dataframe...")
ml_df.drop(["imdb_id"], axis=1, inplace=True)
ml_df[ml_df.isna().any(axis=1)]
ml_df.dropna(inplace=True)

# tt = (
#     ("actors", "actors"),
#     ("titre_genres", "titre_genres"),
#     ("directors", "directors"),
# )
# for t in tt:
#     ml_df[t[0]] = ml_df[t[1]].apply(
#         lambda x: ", ".join(map(str, x))
#     ).replace(" ", "")

# # Full loWer pour reduire les titres, actors, directors etc...
# # for t in tt:
# #     ml_df[t[0]] = ml_df[t[1]].apply(full_lower)
# logging.info(f"Process Overview...")
# ml_df['overview'] = ml_df['overview'].astype(str).apply(clean_overview)

# logging.info(f"Writing {name} dataframe...")
# ml_df.to_parquet(path_file)